## Requirements

### Imports

In [1]:
import os
import shutil
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, OneCycleLR
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
from ray import tune
from torchvision import models
import numpy as np
from torch.utils.tensorboard import SummaryWriter


c:\Users\ilian\Documents\Projects\git_projects\university\mushroom_classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-09 21:45:33,875	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-01-09 21:45:36,673	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
torch.manual_seed(42)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Paths

In [4]:
src_folder = Path.cwd()
root_folder = src_folder.parent
dataset_folder = os.path.join(root_folder, 'dataset')
models_folder = os.path.join(root_folder, 'models')

baseline_folder = os.path.join(models_folder, 'baselines')
result_folder = os.path.join(baseline_folder, 'results')

PREPROCESSED_DIR = os.path.join(dataset_folder, 'preprocessed')
CSV_PATH = os.path.join(dataset_folder, 'csv_mappings', 'train.csv')

## Config

In [5]:
BATCH_SIZE = 32
NUM_CLASSES = 10 
EPOCHS = 20
PATIENCE = 3
LEARNING_RATE = 0.0001

## Data Preperation

### Mushroom Dataset

In [6]:
class MushroomDataset(Dataset):
    def __init__(self, preprocessed_dir, csv_path, transform=None):
        self.preprocessed_dir = preprocessed_dir  
        self.csv_path = csv_path  
        self.transform = transform  
        self.csv_data = pd.read_csv(csv_path)
        
        # Images and Labels
        self.image_ids = self.csv_data['Image'].values  
        self.labels = self.csv_data['Mushroom'].values 
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        label = self.labels[idx]

        image_id_str = str(image_id).zfill(5)  # Pad for filename
        
        # Load .pt files
        image_path = os.path.join(self.preprocessed_dir, f"{image_id_str}.pt")
        image = torch.load(image_path)  
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


### Prepare data

In [7]:
dataset = MushroomDataset(PREPROCESSED_DIR, CSV_PATH)
indices = list(range(len(dataset)))

### Split data

In [8]:
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

In [9]:
train_subset = torch.utils.data.Subset(dataset, train_indices)
val_subset = torch.utils.data.Subset(dataset, val_indices)
test_subset = torch.utils.data.Subset(dataset, test_indices)

### Dataloaders

In [10]:
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False)

## Train & Test functions

In [11]:
def save_model(model, optimizer, epoch, loss, accuracy, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'accuracy': accuracy,
    }

    torch.save(checkpoint, file_path)
    print(f"Model saved to {file_path}")

In [12]:
def train_on_epoch(model, train_loader, criterion, optimizer, device, epoch, writer, scheduler):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, data in enumerate(tqdm(train_loader, desc="[Train]")):
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100.0 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    for param_group in optimizer.param_groups:
        writer.add_scalar('Train/Learning Rate', param_group['lr'], epoch)

    if scheduler:
        scheduler.step()

    writer.add_scalar('Train/Loss', avg_train_loss, epoch)
    writer.add_scalar('Train/Accuracy', train_accuracy, epoch)

    return avg_train_loss, train_accuracy

In [13]:
def validate_on_epoch(model, val_loader, criterion, optimizer, device, epoch, writer, best_val_loss, patience, epochs_no_improve, save_path):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, data in enumerate(tqdm(val_loader, desc="[Val]")):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_accuracy = 100.0 * correct / total
    avg_val_loss = val_loss / len(val_loader)

    writer.add_scalar('Validation/Loss', avg_val_loss, epoch)
    writer.add_scalar('Validation/Accuracy', val_accuracy, epoch)

    # Early stopping 
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        save_model(model, optimizer, epoch, avg_val_loss, val_accuracy, save_path)

    return avg_val_loss, val_accuracy

In [14]:
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for data in tqdm(test_loader, desc="[Test]"):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # Store preds and labels for plots
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_accuracy = 100.0 * correct / total
    avg_test_loss = test_loss / len(test_loader)

    print(f"Test Loss = {avg_test_loss:.4f}")
    print(f"Test Accuracy = {test_accuracy:.2f}%")
    
    return avg_test_loss, test_accuracy, all_labels, all_predictions


In [15]:
def plot_confusion_matrix(all_labels, all_predictions, num_classes, save_path=None):
    conf_matrix = confusion_matrix(all_labels, all_predictions, labels=np.arange(num_classes))
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
                xticklabels=np.arange(num_classes), yticklabels=np.arange(num_classes))
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
        print(f"Confusion matrix saved to {save_path}")

    plt.show()


In [16]:
def per_class_accuracy(all_labels, all_predictions, num_classes):
    class_accuracies = []
    
    for i in range(num_classes):
        class_indices = [j for j, label in enumerate(all_labels) if label == i]
        class_predictions = [all_predictions[j] for j in class_indices]
        class_labels = [all_labels[j] for j in class_indices]
        
        class_accuracy = accuracy_score(class_labels, class_predictions)
        class_accuracies.append(class_accuracy)
        print(f"Accuracy class {i}: {class_accuracy:.4f}")
    
    return class_accuracies


In [17]:
def display_classification_report_as_dataframe(all_labels, all_predictions):
    report_dict = classification_report(all_labels, all_predictions, target_names=[str(i) for i in range(len(set(all_labels)))], output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    display(report_df)
    return report_df

# Training

#### Custom Model

In [18]:
class EnhancedResNet(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0.5):
        super(EnhancedResNet, self).__init__()

        self.resnet = models.resnet50(pretrained=True)  
        self.resnet.fc = nn.Identity()
        self.dropout = nn.Dropout(p=dropout_prob)
        
        self.fc1 = nn.Linear(2048, 1024)  
        self.fc2 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.resnet(x)  
        x = self.dropout(x) 
        x = torch.relu(self.fc1(x)) 
        x = self.fc2(x)  
        return x


In [19]:
from torch.optim.lr_scheduler import OneCycleLR

def get_optimizer_and_scheduler(model, lr=0.001, weight_decay=1e-5, momentum=0.9):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # scheduler = StepLR(optimizer, step_size=2, gamma=0.5)
    scheduler = OneCycleLR(
        optimizer,
        max_lr=lr,  
        steps_per_epoch=len(train_loader),
        epochs=EPOCHS,
        pct_start=0.3,
        anneal_strategy='cos',
        div_factor=25.0,
        final_div_factor=10000.0
    )
    
    return optimizer, scheduler

#### Setter for model 

In [20]:
def set_model_for_training(config, model_type, baseline_folder, NUM_CLASSES):
    base_log_path = os.path.join(baseline_folder, model_type, 'log')
    base_result_path = os.path.join(baseline_folder, model_type, 'results')

    if os.path.exists(base_log_path):
        shutil.rmtree(base_log_path)
    os.makedirs(base_log_path, exist_ok=True)
    os.makedirs(base_result_path, exist_ok=True)

    dropout_prob = config['dropout_prob']
    model = EnhancedResNet(num_classes=NUM_CLASSES, dropout_prob=dropout_prob)

    save_path = os.path.join(base_result_path, "model_custom.pth")
    writer = SummaryWriter(log_dir=base_log_path)

    return model, save_path, writer


In [21]:
def objective(config, train_loader, val_loader, device, baseline_folder, NUM_CLASSES, EPOCHS):
    model, save_path, writer = set_model_for_training(config, model_type='custom', baseline_folder=baseline_folder, NUM_CLASSES=NUM_CLASSES)
    optimizer, scheduler = get_optimizer_and_scheduler(model, config, train_loader, EPOCHS)

    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    epochs_no_improve = 0
    patience = 5

    for epoch in range(EPOCHS):
        # Train and validate
        train_loss, train_accuracy = train_on_epoch(model, train_loader, criterion, optimizer, device, epoch, writer, scheduler)
        val_loss, val_accuracy = validate_on_epoch(model, val_loader, criterion, optimizer, device, epoch, writer, best_val_loss, patience, epochs_no_improve, save_path)
        
        # Report Ray Tune
        tune.report(val_loss=val_loss, val_accuracy=val_accuracy)

        # Early stopping
        if epochs_no_improve >= patience:
            break
    
    return val_loss


## Training

In [24]:
import os
import shutil
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from torch.optim.lr_scheduler import OneCycleLR


torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
src_folder = Path.cwd()
root_folder = src_folder.parent
dataset_folder = os.path.join(root_folder, 'dataset')
models_folder = os.path.join(root_folder, 'models')

custom_folder = os.path.join(models_folder, 'custom')
result_folder = os.path.join(custom_folder, 'results')

PREPROCESSED_DIR = os.path.join(dataset_folder, 'preprocessed')
CSV_PATH = os.path.join(dataset_folder, 'csv_mappings', 'train.csv')

BATCH_SIZE = 32
NUM_CLASSES = 10
EPOCHS = 20
PATIENCE = 3

class MushroomDataset(Dataset):
    def __init__(self, preprocessed_dir, csv_path, transform=None):
        self.preprocessed_dir = preprocessed_dir  
        self.csv_path = csv_path  
        self.transform = transform  
        self.csv_data = pd.read_csv(csv_path)
        
        self.image_ids = self.csv_data['Image'].values  
        self.labels = self.csv_data['Mushroom'].values 
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        label = self.labels[idx]

        image_id_str = str(image_id).zfill(5)  
        
        image_path = os.path.join(self.preprocessed_dir, f"{image_id_str}.pt")
        image = torch.load(image_path)  
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

dataset = MushroomDataset(PREPROCESSED_DIR, CSV_PATH)
indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)
train_subset = torch.utils.data.Subset(dataset, train_indices)
val_subset = torch.utils.data.Subset(dataset, val_indices)
test_subset = torch.utils.data.Subset(dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False)

def save_model(model, optimizer, epoch, loss, accuracy, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'accuracy': accuracy,
    }
    torch.save(checkpoint, file_path)
    print(f"Model saved to {file_path}")

def train_on_epoch(model, train_loader, criterion, optimizer, device, epoch, writer, scheduler):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, data in enumerate(tqdm(train_loader, desc="[Train]")):
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        batch_accuracy = 100.0 * correct / total
        writer.add_scalar('Train/Loss', loss.item(), epoch * len(train_loader) + batch_idx)
        writer.add_scalar('Train/Accuracy', batch_accuracy, epoch * len(train_loader) + batch_idx)

    train_accuracy = 100.0 * correct / total
    avg_train_loss = train_loss / len(train_loader)

    for param_group in optimizer.param_groups:
        writer.add_scalar('Train/Learning Rate', param_group['lr'], epoch)

    if scheduler:
        scheduler.step()

    return avg_train_loss, train_accuracy

def validate_on_epoch(model, val_loader, criterion, optimizer, device, epoch, writer, best_val_loss, patience, epochs_no_improve, save_path):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, data in enumerate(tqdm(val_loader, desc="[Val]")):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            batch_accuracy = 100.0 * correct / total
            writer.add_scalar('Validation/Loss', loss.item(), epoch * len(val_loader) + batch_idx)
            writer.add_scalar('Validation/Accuracy', batch_accuracy, epoch * len(val_loader) + batch_idx)

    val_accuracy = 100.0 * correct / total
    avg_val_loss = val_loss / len(val_loader)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        save_model(model, optimizer, epoch, avg_val_loss, val_accuracy, save_path)
        print(f"Model saved to {save_path}")
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered at epoch {epoch + 1}")
        return best_val_loss, epochs_no_improve, True, val_accuracy 

    return best_val_loss, epochs_no_improve, False, val_accuracy

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs, device, writer, scheduler, patience, save_path):
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        train_loss, train_accuracy = train_on_epoch(model, train_loader, criterion, optimizer, device, epoch, writer, scheduler)
        print(f"Train Loss = {train_loss:.4f}, Train Acc = {train_accuracy:.2f}%")

        best_val_loss, epochs_no_improve, early_stop, val_accuracy = validate_on_epoch(
            model, val_loader, criterion, optimizer, device, epoch, writer, best_val_loss, patience, epochs_no_improve, save_path
        )
        print(f"Val Loss = {best_val_loss:.4f}, Val Acc = {val_accuracy:.2f}%")

        if early_stop:
            break  

    return model

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for data in tqdm(test_loader, desc="[Test]"):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    test_accuracy = 100.0 * correct / total
    avg_test_loss = test_loss / len(test_loader)

    print(f"Test Loss = {avg_test_loss:.4f}")
    print(f"Test Accuracy = {test_accuracy:.2f}%")
    
    return avg_test_loss, test_accuracy, all_labels, all_predictions

def plot_confusion_matrix(all_labels, all_predictions, num_classes, save_path=None):
    conf_matrix = confusion_matrix(all_labels, all_predictions, labels=np.arange(num_classes))
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
                xticklabels=np.arange(num_classes), yticklabels=np.arange(num_classes))
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
        print(f"Confusion matrix saved to {save_path}")

    plt.show()

def per_class_accuracy(all_labels, all_predictions, num_classes):
    class_accuracies = []
    
    for i in range(num_classes):
        class_indices = [j for j, label in enumerate(all_labels) if label == i]
        class_predictions = [all_predictions[j] for j in class_indices]
        class_labels = [all_labels[j] for j in class_indices]
        
        class_accuracy = accuracy_score(class_labels, class_predictions)
        class_accuracies.append(class_accuracy)
        print(f"Accuracy class {i}: {class_accuracy:.4f}")
    
    return class_accuracies

def display_classification_report_as_dataframe(all_labels, all_predictions):
    report_dict = classification_report(all_labels, all_predictions, target_names=[str(i) for i in range(len(set(all_labels)))], output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    display(report_df)
    return report_df

class EnhancedResNet(nn.Module):
    def __init__(self, num_classes=10, dropout_prob=0.5):
        super(EnhancedResNet, self).__init__()

        self.resnet = models.resnet50(pretrained=True)  
        self.resnet.fc = nn.Identity()
        self.dropout = nn.Dropout(p=dropout_prob)
        
        self.fc1 = nn.Linear(2048, 1024)  
        self.fc2 = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.resnet(x)  
        x = self.dropout(x) 
        x = torch.relu(self.fc1(x)) 
        x = self.fc2(x)  
        return x

def get_optimizer_and_scheduler(model, lr=0.001, weight_decay=1e-5, momentum=0.9):
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    scheduler = OneCycleLR(
        optimizer,
        max_lr=lr,  
        steps_per_epoch=len(train_loader),
        epochs=EPOCHS,
        pct_start=0.3,
        anneal_strategy='cos',
        div_factor=25.0,
        final_div_factor=10000.0
    )
    
    return optimizer, scheduler

def set_model_for_training(model_type, dropout_prob=0.5):
    base_log_path = os.path.join(custom_folder, model_type, 'log')
    base_result_path = os.path.join(custom_folder, model_type, 'results')

    if os.path.exists(base_log_path):
        shutil.rmtree(base_log_path)
    os.makedirs(base_log_path, exist_ok=True)
    os.makedirs(base_result_path, exist_ok=True)

    if model_type == 'custom':
        model = EnhancedResNet(num_classes=NUM_CLASSES, dropout_prob=dropout_prob)
        save_path = os.path.join(base_result_path, "model_custom.pth")

    else:
        raise ValueError(f"Unsupported model type")
    
    writer = SummaryWriter(log_dir=base_log_path)

    return model, save_path, writer

def train_tune(config):
    model_type = 'custom'
    model, save_path, writer = set_model_for_training(model_type, dropout_prob=config["dropout_prob"])
    criterion = nn.CrossEntropyLoss()
    optimizer, scheduler = get_optimizer_and_scheduler(model, lr=config["lr"])
    model = train_and_validate(model, 
                               train_loader, 
                               val_loader, 
                               criterion, 
                               optimizer, 
                               epochs=EPOCHS, 
                               device=device, 
                               writer=writer, 
                               scheduler=scheduler, 
                               patience=PATIENCE, 
                               save_path=save_path)
    avg_test_loss, test_accuracy, all_labels, all_predictions = evaluate_model(model, test_loader, criterion, device)
    return test_accuracy

search_space = {
    "lr": tune.grid_search([0.0001, 0.001, 0.01]),
    "dropout_prob": tune.grid_search([0.3, 0.5, 0.7])
}

scheduler = ASHAScheduler(
    metric="accuracy",
    mode="max",
    max_t=EPOCHS,
    grace_period=1,
    reduction_factor=2)

analysis = tune.run(
    train_tune,
    config=search_space,
    scheduler=scheduler,
    num_samples=1,
    resources_per_trial={"cpu": 1, "gpu": 1}
)


2025-01-09 21:51:22,650	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2025-01-09 21:52:23,150	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 1.0 CPUs and 1.0 GPUs per trial, but the cluster only has 22.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.
2025-01-09 21:53:23,226	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 1.0 CPUs and 1.0 GPUs per trial, but the cluster only has 22.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via th